In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
!pip install optuna schedulefree -q

In [ ]:
train=pd.read_csv("/kaggle/input/eds-232-ocean-chemistry-prediction-for-calcofi/train.csv")
test=pd.read_csv("/kaggle/input/eds-232-ocean-chemistry-prediction-for-calcofi/test.csv")
sub=pd.read_csv("/kaggle/input/eds-232-ocean-chemistry-prediction-for-calcofi/sample_submission.csv")

In [ ]:
import torch
import torch.nn as nn
from sklearn.model_selection import KFold
from sklearn.preprocessing import RobustScaler, PolynomialFeatures
from sklearn.neighbors import NearestNeighbors
import numpy as np
import random

# Set random seeds for reproducibility
SEED = 42
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

# Drop unneeded columns and handle missing values
train = train.drop(columns=["Unnamed: 12", "id"])  # Dropping unnecessary columns

# CRITICAL FIX: Rename TA1.x to TA1 to match test data
train = train.rename(columns={"TA1.x": "TA1"})

# ============================================
# Feature Engineering
# ============================================
print("="*60)
print("FEATURE ENGINEERING")
print("="*60)

# Extract features and target
feature_columns = [col for col in train.columns if col != 'DIC']
X = train[feature_columns].copy()
y = train['DIC'].values
X_test = test[feature_columns].copy()

print(f"\nOriginal features: {X.shape[1]} features")

# Generate polynomial features (degree=2) including interaction terms
# This will create: original features + squared terms + all cross-product terms
poly = PolynomialFeatures(degree=2, include_bias=False)

print(f"\nGenerating polynomial features (degree=2)...")
print(f"  - Original features")
print(f"  - Squared terms (x^2)")
print(f"  - Interaction terms (x_i * x_j)")

X = poly.fit_transform(X)
X_test = poly.transform(X_test)

print(f"\nPolynomial features generated:")
print(f"  Original: 16 features")
print(f"  After polynomial expansion: {X.shape[1]} features")
print(f"  (16 original + 16 squared + 120 interaction terms)")

# ============================================
# Robust Scaling + Smooth Clipping
# ============================================
print(f"\n{'='*60}")
print("ROBUST SCALING + SMOOTH CLIPPING")
print(f"{'='*60}")

# Use RobustScaler (median and IQR-based, robust to outliers)
scaler_X = RobustScaler()
scaler_y = RobustScaler()

X_scaled = scaler_X.fit_transform(X)
X_test_scaled = scaler_X.transform(X_test)
y_scaled = scaler_y.fit_transform(y.reshape(-1, 1)).flatten()

print(f"RobustScaler applied (median-based, robust to outliers)")

# Smooth clipping using tanh (softer than hard clipping)
clip_value = 3.0  # Soft clip at ±3 (similar to ±3 std in StandardScaler)

def smooth_clip(x, clip_val=3.0):
    """
    Smooth clipping using tanh function
    - Values around 0 are unchanged
    - Large values are smoothly compressed towards ±clip_val
    """
    return np.tanh(x / clip_val) * clip_val

X_scaled = smooth_clip(X_scaled, clip_value)
X_test_scaled = smooth_clip(X_test_scaled, clip_value)
y_scaled = smooth_clip(y_scaled, clip_value)

print(f"Smooth clipping applied (tanh-based, clip_value={clip_value})")
print(f"  X_scaled range: [{X_scaled.min():.3f}, {X_scaled.max():.3f}]")
print(f"  y_scaled range: [{y_scaled.min():.3f}, {y_scaled.max():.3f}]")

print(f"\nNormalized features:")
print(f"  X_scaled shape: {X_scaled.shape}")
print(f"  y_scaled shape: {y_scaled.shape}")

# ============================================
# K-Fold Cross Validation Setup
# ============================================
print(f"\n{'='*60}")
print("K-FOLD CROSS VALIDATION SETUP")
print(f"{'='*60}")

n_splits = 5  # Number of folds
kfold = KFold(n_splits=n_splits, shuffle=True, random_state=SEED)

print(f"K-Fold CV: {n_splits} folds")
print(f"Each fold will be trained separately")
print(f"Final prediction: ensemble of all {n_splits} models")
print("="*60)

In [ ]:
from torch.utils.data import Dataset, DataLoader

class OceanChemistryDataset(Dataset):
    def __init__(self, X, y):
        self.X = torch.tensor(X, dtype=torch.float32)
        # Handle both numpy arrays and pandas Series
        if hasattr(y, 'values'):
            y = y.values
        self.y = torch.tensor(y, dtype=torch.float32)

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

train_dataset = OceanChemistryDataset(X_train, y_train)
val_dataset = OceanChemistryDataset(X_val, y_val)

# Set generator for reproducible shuffling
g = torch.Generator()
g.manual_seed(SEED)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True, generator=g)
val_loader = DataLoader(val_dataset, batch_size=64, shuffle=False)

print(f"Training samples: {len(train_dataset)}")
print(f"Validation samples: {len(val_dataset)}")

In [ ]:
class MLPModel(nn.Module):
    def __init__(self, input_size, hidden_sizes=[128], dropout_rate=0.0, activation='relu', use_batchnorm=True):
        """
        MLP model with flexible number of hidden layers
        
        Args:
            input_size: Number of input features
            hidden_sizes: List of hidden layer sizes (e.g., [128], [128, 64], [256, 128, 64])
            dropout_rate: Dropout probability
            activation: Activation function name
            use_batchnorm: Whether to use batch normalization
        """
        super(MLPModel, self).__init__()

        self.use_batchnorm = use_batchnorm
        self.num_layers = len(hidden_sizes)

        # Activation function mapping
        activation_map = {
            'relu': nn.ReLU(),
            'leaky_relu': nn.LeakyReLU(0.1),
            'elu': nn.ELU(),
            'gelu': nn.GELU(),
            'silu': nn.SiLU(),  # Swish
            'tanh': nn.Tanh()
        }

        # Select activation function
        self.activation = activation_map.get(activation, nn.ReLU())

        # Determine initialization based on activation
        nonlinearity = 'relu' if activation in ['relu', 'leaky_relu'] else 'linear'

        # Build hidden layers dynamically
        self.hidden_layers = nn.ModuleList()
        self.batch_norms = nn.ModuleList() if use_batchnorm else None
        self.dropouts = nn.ModuleList()
        
        layer_sizes = [input_size] + hidden_sizes
        
        for i in range(len(hidden_sizes)):
            # Hidden layer
            layer = nn.Linear(layer_sizes[i], layer_sizes[i+1])
            if nonlinearity == 'relu':
                nn.init.kaiming_normal_(layer.weight, mode='fan_in', nonlinearity='relu')
            else:
                nn.init.xavier_normal_(layer.weight)
            nn.init.constant_(layer.bias, 0)
            self.hidden_layers.append(layer)
            
            # Batch normalization
            if use_batchnorm:
                self.batch_norms.append(nn.BatchNorm1d(layer_sizes[i+1]))
            
            # Dropout
            self.dropouts.append(nn.Dropout(dropout_rate))
        
        # Output layer
        self.output = nn.Linear(hidden_sizes[-1], 1)
        nn.init.xavier_normal_(self.output.weight)
        nn.init.constant_(self.output.bias, 0)
        
        # Residual connection (shortcut from input to output)
        self.shortcut = nn.Linear(input_size, 1)
        nn.init.xavier_normal_(self.shortcut.weight)
        nn.init.constant_(self.shortcut.bias, 0)

    def forward(self, x):
        # Main path: input -> hidden layers -> output
        h = x
        for i in range(self.num_layers):
            h = self.hidden_layers[i](h)
            if self.use_batchnorm:
                h = self.batch_norms[i](h)
            h = self.activation(h)
            h = self.dropouts[i](h)
        
        main_output = self.output(h)
        
        # Residual path: input -> output (shortcut)
        residual = self.shortcut(x)
        
        # Add residual connection
        output = main_output + residual
        
        return output

# Set device (GPU if available, else CPU)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")

print(f"\nMLP Model Architecture:")
print(f"  Input -> Hidden Layers (configurable) -> Output")
print(f"  + Residual connection (Input -> Output)")
print(f"  Each hidden layer: Linear -> [BatchNorm (optional)] -> Activation -> Dropout")
print(f"  hidden_sizes: List of layer sizes (e.g., [128], [128, 64], [256, 128, 64])")
print(f"  Dropout: Fixed to 0.0")
print(f"  BatchNorm: Can be enabled/disabled")

In [ ]:
import torch.optim as optim
from schedulefree import RAdamScheduleFree, AdamWScheduleFree
from torch.utils.data import Dataset, DataLoader, TensorDataset
import copy

# Hyperparameters
hidden_sizes = [128]
dropout_rate = 0.0
activation = 'gelu'
use_batchnorm = False
loss_function = 'mae'
optimizer_name = 'adamw_schedulefree'
lr = 0.001
weight_decay = 1e-3
batch_size = 64
epochs = 1000
beta1 = 0.9
beta2 = 0.999

# Gaussian Noise Injection
use_gaussian_noise = True
noise_std = 0.05
noise_prob = 0.5

# SMOTER parameters
use_smoter = True
smoter_k = 5
smoter_percentage = 0.2

print("="*60)
print("K-FOLD CROSS VALIDATION TRAINING")
print("="*60)
print(f"Folds: {n_splits}")
print(f"Hidden layers: {hidden_sizes}")
print(f"Activation: {activation}")
print(f"Loss: {loss_function}")
print(f"Optimizer: {optimizer_name}")
print(f"LR: {lr}, Weight decay: {weight_decay}")
print(f"Batch size: {batch_size}, Epochs: {epochs}")
print(f"Gaussian Noise: std={noise_std}, prob={noise_prob}")
print(f"SMOTER: k={smoter_k}, percentage={smoter_percentage:.0%}")
print("="*60)

# Loss function
loss_map = {'mse': nn.MSELoss(), 'mae': nn.L1Loss(), 'smooth_l1': nn.SmoothL1Loss(), 'huber': nn.HuberLoss()}
criterion = loss_map[loss_function]

# Inverse smooth clip function
def inverse_smooth_clip(x, clip_val=3.0):
    x_clipped = np.clip(x / clip_val, -0.9999, 0.9999)
    return np.arctanh(x_clipped) * clip_val

# Store models and metrics for each fold
fold_models = []
fold_metrics = []

# K-Fold Cross Validation Loop
for fold, (train_idx, val_idx) in enumerate(kfold.split(X_scaled)):
    print(f"\n{'='*60}")
    print(f"FOLD {fold + 1}/{n_splits}")
    print(f"{'='*60}")
    
    # Split data
    X_train_fold = X_scaled[train_idx]
    y_train_fold = y_scaled[train_idx]
    X_val_fold = X_scaled[val_idx]
    y_val_fold = y_scaled[val_idx]
    
    print(f"Fold {fold + 1} - Train: {len(X_train_fold)}, Val: {len(X_val_fold)}")
    
    # Apply SMOTER to training fold only
    if use_smoter:
        original_size = len(X_train_fold)
        n_synthetic = int(original_size * smoter_percentage)
        
        knn = NearestNeighbors(n_neighbors=smoter_k + 1, metric='euclidean')
        knn.fit(X_train_fold)
        
        X_synthetic = []
        y_synthetic = []
        
        for _ in range(n_synthetic):
            idx = np.random.randint(0, len(X_train_fold))
            sample_X = X_train_fold[idx]
            sample_y = y_train_fold[idx]
            
            distances, indices = knn.kneighbors([sample_X])
            neighbor_indices = indices[0][1:]
            neighbor_idx = np.random.choice(neighbor_indices)
            neighbor_X = X_train_fold[neighbor_idx]
            neighbor_y = y_train_fold[neighbor_idx]
            
            alpha = np.random.random()
            synthetic_X = sample_X + alpha * (neighbor_X - sample_X)
            synthetic_y = sample_y + alpha * (neighbor_y - sample_y)
            
            X_synthetic.append(synthetic_X)
            y_synthetic.append(synthetic_y)
        
        X_train_fold = np.vstack([X_train_fold, np.array(X_synthetic)])
        y_train_fold = np.concatenate([y_train_fold, np.array(y_synthetic)])
        
        print(f"SMOTER: {original_size} -> {len(X_train_fold)} (+{len(X_synthetic)})")
    
    # Create DataLoaders
    train_dataset = TensorDataset(torch.tensor(X_train_fold, dtype=torch.float32), 
                                   torch.tensor(y_train_fold, dtype=torch.float32))
    val_dataset = TensorDataset(torch.tensor(X_val_fold, dtype=torch.float32), 
                                 torch.tensor(y_val_fold, dtype=torch.float32))
    
    g = torch.Generator().manual_seed(SEED + fold)
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, generator=g)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
    
    # Initialize model
    model = MLPModel(input_size=X_train_fold.shape[1], hidden_sizes=hidden_sizes, 
                     dropout_rate=dropout_rate, activation=activation, use_batchnorm=use_batchnorm)
    model = model.to(device)
    
    # Optimizer
    is_schedulefree = optimizer_name.endswith('_schedulefree')
    if optimizer_name == 'adamw_schedulefree':
        optimizer = AdamWScheduleFree(model.parameters(), lr=lr, betas=(beta1, beta2), weight_decay=weight_decay)
    elif optimizer_name == 'radam_schedulefree':
        optimizer = RAdamScheduleFree(model.parameters(), lr=lr, betas=(beta1, beta2), weight_decay=weight_decay)
    else:
        optimizer = optim.AdamW(model.parameters(), lr=lr, betas=(beta1, beta2), weight_decay=weight_decay)
    
    # Training loop
    best_val_rmse = float('inf')
    best_model_state = None
    
    for epoch in range(epochs):
        if is_schedulefree:
            optimizer.train()
        
        model.train()
        for X_batch, y_batch in train_loader:
            X_batch, y_batch = X_batch.to(device), y_batch.to(device)
            
            if use_gaussian_noise and np.random.rand() < noise_prob:
                noise = torch.randn_like(X_batch) * noise_std
                X_batch = X_batch + noise
            
            optimizer.zero_grad()
            outputs = model(X_batch)
            loss = criterion(outputs.squeeze(), y_batch)
            loss.backward()
            optimizer.step()
        
        if is_schedulefree:
            optimizer.eval()
        
        # Validation
        model.eval()
        val_predictions = []
        val_targets = []
        with torch.no_grad():
            for X_batch, y_batch in val_loader:
                X_batch, y_batch = X_batch.to(device), y_batch.to(device)
                outputs = model(X_batch)
                val_predictions.extend(outputs.squeeze().cpu().numpy())
                val_targets.extend(y_batch.cpu().numpy())
        
        val_predictions_unclipped = inverse_smooth_clip(np.array(val_predictions), clip_value)
        val_targets_unclipped = inverse_smooth_clip(np.array(val_targets), clip_value)
        
        val_predictions_original = scaler_y.inverse_transform(val_predictions_unclipped.reshape(-1, 1)).flatten()
        val_targets_original = scaler_y.inverse_transform(val_targets_unclipped.reshape(-1, 1)).flatten()
        
        val_rmse = np.sqrt(np.mean((val_predictions_original - val_targets_original)**2))
        
        if val_rmse < best_val_rmse:
            best_val_rmse = val_rmse
            best_model_state = copy.deepcopy(model.state_dict())
        
        if epoch % 200 == 0:
            print(f"Fold {fold + 1} - Epoch {epoch + 1}/{epochs} | Val RMSE: {val_rmse:.4f} | Best: {best_val_rmse:.4f}")
    
    # Load best model
    model.load_state_dict(best_model_state)
    fold_models.append(model)
    fold_metrics.append(best_val_rmse)
    
    print(f"Fold {fold + 1} complete - Best Val RMSE: {best_val_rmse:.4f}")

# Summary
print(f"\n{'='*60}")
print("K-FOLD CV SUMMARY")
print(f"{'='*60}")
for i, rmse in enumerate(fold_metrics):
    print(f"Fold {i + 1}: {rmse:.4f}")
print(f"Mean CV RMSE: {np.mean(fold_metrics):.4f} ± {np.std(fold_metrics):.4f}")
print(f"{'='*60}")

In [ ]:
# ============================================
# Optuna Hyperparameter Optimization
# ============================================

import optuna
from optuna.trial import Trial
from optuna.pruners import HyperbandPruner
from optuna.samplers import TPESampler
from schedulefree import RAdamScheduleFree, AdamWScheduleFree
import copy

# Inverse smooth clip function for Optuna
def inverse_smooth_clip_optuna(x, clip_val=3.0):
    """
    Inverse of smooth clipping (arctanh)
    """
    # Clip to prevent arctanh overflow
    x_clipped = np.clip(x / clip_val, -0.9999, 0.9999)
    return np.arctanh(x_clipped) * clip_val

def objective(trial: Trial):
    # Number of layers selection (1, 2, or 3 layers)
    num_layers = trial.suggest_int('num_layers', 1, 3)
    
    # Build hidden_sizes list based on num_layers
    hidden_sizes = []
    for i in range(num_layers):
        layer_size = trial.suggest_categorical(f'hidden_size_layer_{i+1}', [32, 64, 128, 256, 512, 1024])
        hidden_sizes.append(layer_size)
    
    # Optimizer selection: AdamWScheduleFree or RAdamScheduleFree only
    optimizer_name = trial.suggest_categorical('optimizer', ['adamw_schedulefree', 'radam_schedulefree'])
    
    # NO scheduler exploration (using SchedulerFree optimizers)
    
    # Hyperparameters to optimize
    lr = trial.suggest_float('lr', 1e-5, 1e-2, log=True)
    weight_decay = trial.suggest_float('weight_decay', 1e-5, 1e-2, log=True)
    dropout_rate = 0.0  # Fixed to 0
    batch_size = trial.suggest_categorical('batch_size', [16, 32, 64, 128, 256])
    
    # Batch Normalization: fixed to False
    use_batchnorm = False
    
    # Adam betas (momentum coefficients)
    beta1 = trial.suggest_float('beta1', 0.85, 0.95)
    beta2 = trial.suggest_float('beta2', 0.9, 0.9999)
    
    # Activation function selection
    activation = trial.suggest_categorical('activation', ['relu', 'gelu', 'silu'])
    
    # Loss function: MAE fixed
    loss_function = 'mae'
    
    # Gaussian Noise parameters (always enabled)
    use_gaussian_noise = True
    noise_std = trial.suggest_float('noise_std', 0.01, 0.15)
    noise_prob = trial.suggest_float('noise_prob', 0.3, 0.9)
    
    # Loss criterion (MAE fixed)
    criterion_trial = nn.L1Loss()
    
    # Create dataloaders with suggested batch size
    g_trial = torch.Generator()
    g_trial.manual_seed(SEED)
    train_loader_trial = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, generator=g_trial)
    val_loader_trial = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
    
    # Create model with hidden_sizes (list)
    model_trial = MLPModel(input_size=X_train.shape[1], hidden_sizes=hidden_sizes, 
                          dropout_rate=dropout_rate, activation=activation, use_batchnorm=use_batchnorm)
    model_trial = model_trial.to(device)
    
    # Create optimizer based on selection (SchedulerFree only)
    if optimizer_name == 'adamw_schedulefree':
        optimizer_trial = AdamWScheduleFree(
            model_trial.parameters(), 
            lr=lr, 
            betas=(beta1, beta2),
            weight_decay=weight_decay
        )
    elif optimizer_name == 'radam_schedulefree':
        optimizer_trial = RAdamScheduleFree(
            model_trial.parameters(), 
            lr=lr, 
            betas=(beta1, beta2),
            weight_decay=weight_decay
        )
    
    # No scheduler (using SchedulerFree optimizers)
    scheduler = None
    
    # Early stopping
    best_val_rmse = float('inf')
    best_model_state = None  # Store best model parameters
    patience = 200
    patience_counter = 0
    
    # Training loop
    max_epochs = 1000
    for epoch in range(max_epochs):
        # Set optimizer to train mode (important for SchedulerFree)
        optimizer_trial.train()
        
        # Training
        model_trial.train()
        for X_batch, y_batch in train_loader_trial:
            X_batch, y_batch = X_batch.to(device), y_batch.to(device)
            
            # Apply Gaussian Noise Injection (always enabled)
            if np.random.rand() < noise_prob:
                noise = torch.randn_like(X_batch) * noise_std
                X_batch_noisy = X_batch + noise
            else:
                X_batch_noisy = X_batch
            
            optimizer_trial.zero_grad()
            outputs = model_trial(X_batch_noisy)
            loss = criterion_trial(outputs.squeeze(), y_batch)
            loss.backward()
            optimizer_trial.step()
        
        # Set optimizer to eval mode (uses averaged weights for SchedulerFree)
        optimizer_trial.eval()
        
        # Validation
        model_trial.eval()
        val_predictions = []
        val_targets = []
        with torch.no_grad():
            for X_batch, y_batch in val_loader_trial:
                X_batch, y_batch = X_batch.to(device), y_batch.to(device)
                outputs = model_trial(X_batch)
                val_predictions.extend(outputs.squeeze().cpu().numpy())
                val_targets.extend(y_batch.cpu().numpy())
        
        # Inverse transform to original scale
        # Step 1: Inverse smooth clip (arctanh)
        val_predictions_unclipped = inverse_smooth_clip_optuna(np.array(val_predictions), clip_value)
        val_targets_unclipped = inverse_smooth_clip_optuna(np.array(val_targets), clip_value)
        
        # Step 2: Inverse RobustScaler transform
        val_predictions_original = scaler_y.inverse_transform(val_predictions_unclipped.reshape(-1, 1)).flatten()
        val_targets_original = scaler_y.inverse_transform(val_targets_unclipped.reshape(-1, 1)).flatten()
        
        # Calculate RMSE in original scale
        val_rmse = np.sqrt(np.mean((val_predictions_original - val_targets_original)**2))
        
        # Early stopping and save best model
        if val_rmse < best_val_rmse:
            best_val_rmse = val_rmse
            best_model_state = copy.deepcopy(model_trial.state_dict())  # Save best model parameters
            patience_counter = 0
        else:
            patience_counter += 1
            if patience_counter >= patience:
                break
        
        # Report intermediate value for Hyperband pruning
        trial.report(val_rmse, epoch)
        
        # Handle pruning
        if trial.should_prune():
            raise optuna.TrialPruned()
    
    # Load best model parameters before returning
    if best_model_state is not None:
        model_trial.load_state_dict(best_model_state)
    
    return best_val_rmse

# Create Sampler with seed for reproducibility
sampler = TPESampler(seed=SEED)

# Create Pruner
pruner = HyperbandPruner(
    min_resource=50,
    max_resource=1000,
    reduction_factor=3
)

# Create study with sampler and pruner
study = optuna.create_study(
    direction='minimize', 
    study_name='mlp_optimization',
    sampler=sampler,  # Sampler with seed for reproducibility
    pruner=pruner
)

print("="*60)
print("Starting Optuna optimization (Focused Search)")
print("="*60)
print(f"Random seed: {SEED}")
print(f"Sampler: TPESampler (seed={SEED})")
print(f"Pruner: HyperbandPruner (min_resource=50, max_resource=1000, reduction_factor=3)")
print(f"\nOptimizing: num_layers (1-3),")
print(f"            hidden_size for each layer (32/64/128/256/512/1024),")
print(f"            optimizer (AdamWScheduleFree/RAdamScheduleFree ONLY),")
print(f"            NO scheduler exploration (using SchedulerFree),")
print(f"            lr, weight_decay,")
print(f"            batch_size (16/32/64/128/256),")
print(f"            beta1 (0.85-0.95), beta2 (0.9-0.9999),")
print(f"            activation (relu/gelu/silu),")
print(f"            gaussian_noise params (std: 0.01-0.15, prob: 0.3-0.9)")
print(f"Loss function: MAE (FIXED)")
print(f"Batch Normalization: FALSE (FIXED)")
print(f"Gaussian Noise: ENABLED (FIXED)")
print(f"Model: Multi-layer MLP with Residual Connection")
print(f"Input: Polynomial features (degree=2, 152 dimensions)")
print(f"Dropout: Fixed to 0.0")
print(f"Scaling: RobustScaler + Smooth Clipping (tanh)")
print(f"Best model checkpoint: Enabled (saves best validation RMSE)")
print(f"Early stopping patience: 200 epochs")
print(f"Max epochs: 1000")
print("="*60)

study.optimize(objective, n_trials=100, timeout=10800)  # 100 trials or 3 hours

# Print best parameters
print("\n" + "="*60)
print("OPTIMIZATION COMPLETE!")
print("="*60)
print(f"\nBest trial:")
print(f"  RMSE: {study.best_trial.value:.4f}")
print(f"\n  Best Parameters:")
for key, value in study.best_trial.params.items():
    print(f"    {key}: {value}")

# Print pruning statistics
pruned_trials = [t for t in study.trials if t.state == optuna.trial.TrialState.PRUNED]
complete_trials = [t for t in study.trials if t.state == optuna.trial.TrialState.COMPLETE]
print(f"\n  Statistics:")
print(f"    Completed trials: {len(complete_trials)}")
print(f"    Pruned trials: {len(pruned_trials)}")
print(f"    Total trials: {len(study.trials)}")
print(f"    Pruning efficiency: {len(pruned_trials)/len(study.trials)*100:.1f}%")
print("="*60)

In [ ]:
# ============================================
# K-Fold Ensemble Prediction with TTA
# ============================================

# TTA parameters
tta_iterations = 20
tta_noise_std = 0.03

print("="*60)
print("K-FOLD ENSEMBLE PREDICTION WITH TTA")
print("="*60)
print(f"Number of models: {len(fold_models)}")
print(f"TTA iterations: {tta_iterations}")
print(f"TTA noise std: {tta_noise_std}")
print("="*60)

# Convert test data to tensor
test_tensor = torch.tensor(X_test_scaled, dtype=torch.float32).to(device)

# Collect predictions from all folds and TTA iterations
all_fold_predictions = []

for fold_idx, model in enumerate(fold_models):
    print(f"\nFold {fold_idx + 1}/{len(fold_models)} predicting...")
    model.eval()
    
    # TTA for this fold
    fold_tta_predictions = []
    
    with torch.no_grad():
        for tta_iter in range(tta_iterations):
            # Add noise for TTA (except first iteration)
            if tta_iter == 0:
                test_tensor_augmented = test_tensor
            else:
                noise = torch.randn_like(test_tensor) * tta_noise_std
                test_tensor_augmented = test_tensor + noise
            
            # Make predictions
            predictions_scaled = model(test_tensor_augmented).squeeze().cpu().numpy()
            
            # Inverse transform
            predictions_unclipped = inverse_smooth_clip(predictions_scaled, clip_value)
            predictions = scaler_y.inverse_transform(predictions_unclipped.reshape(-1, 1)).flatten()
            
            fold_tta_predictions.append(predictions)
    
    # Average TTA predictions for this fold
    fold_avg_prediction = np.mean(fold_tta_predictions, axis=0)
    all_fold_predictions.append(fold_avg_prediction)
    
    print(f"Fold {fold_idx + 1} complete - Prediction range: [{fold_avg_prediction.min():.2f}, {fold_avg_prediction.max():.2f}]")

# Ensemble: Average predictions across all folds
final_predictions = np.mean(all_fold_predictions, axis=0)

# Calculate prediction statistics
fold_std = np.std(all_fold_predictions, axis=0)

print(f"\n{'='*60}")
print("ENSEMBLE COMPLETE")
print(f"{'='*60}")
print(f"Final predictions - Min: {final_predictions.min():.2f}, Max: {final_predictions.max():.2f}, Mean: {final_predictions.mean():.2f}")
print(f"Average std across folds: {fold_std.mean():.4f}")
print(f"(Low std = high agreement between folds)")
print(f"{'='*60}")

# Prepare submission
submission = pd.DataFrame({"id": range(1455, 1455 + len(final_predictions)), "DIC": final_predictions})
submission.to_csv("submission.csv", index=False)
print("\nSubmission saved!")